In [21]:
import os
import json
import yaml
import torch
from datetime import datetime
from typing import Dict
from data_preprocessing import DatasetConfig, DataModule, ModelType
from ensemble_model import create_ensemble_model
from ensemble_trainer import EnsembleTrainer
from evaluation import EnsembleEvaluator
from data_analyser import DataAnalyzer
import wandb


In [2]:

class NumpyEncoder(json.JSONEncoder):
    """Custom encoder for numpy data types"""
    def default(self, obj):
        import numpy as np
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
                          np.int16, np.int32, np.int64, np.uint8,
                          np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, np.float64)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        elif isinstance(obj, np.bool_):
            return bool(obj)
        return json.JSONEncoder.default(self, obj)

def validate_config(config: Dict):
    """Validate configuration"""
    required_keys = [
        'data_path',
        'model',
        'training',
        'evaluation',
        'logging'
    ]
    
    for key in required_keys:
        if key not in config:
            raise ValueError(f"Missing required config key: {key}")
    
    # Validate model config
    model_required = ['num_classes', 'cnn_config', 'vit_config']
    for key in model_required:
        if key not in config['model']:
            raise ValueError(f"Missing required model config key: {key}")
        
def load_config(config_path: str) -> Dict:
    """Load and validate configuration"""
    try:
        with open(config_path, 'r') as f:
            config = yaml.safe_load(f)
        
        # Add default values if missing
        config.setdefault('model', {})
        config['model'].setdefault('image_size', 224)
        config.setdefault('training', {})
        config['training'].setdefault('batch_size', 32)
        config['training'].setdefault('num_workers', 4)
        
        validate_config(config)
        return config
    except FileNotFoundError:
        raise FileNotFoundError(f"Configuration file not found at {config_path}")
    except yaml.YAMLError as e:
        raise ValueError(f"Error parsing configuration file: {str(e)}")
    except Exception as e:
        raise Exception(f"Error loading configuration: {str(e)}")


In [3]:

def setup_directories(experiment_name: str, config: Dict) -> Dict:
    """Setup experiment directories"""
    base_dir = os.path.join(config['logging']['save_dir'], experiment_name)
    print(f"Creating experiment directory at {base_dir}")
    dirs = {
        'checkpoints': os.path.join(base_dir, 'checkpoints'),
        'evaluation': os.path.join(base_dir, 'evaluation'),
        'logs': os.path.join(base_dir, 'logs'),
        'visualizations': os.path.join(base_dir, 'visualizations'),
        'predictions': os.path.join(base_dir, 'predictions')
    }
    
    # Create all directories
    for dir_path in dirs.values():
        os.makedirs(dir_path, exist_ok=True)
    
    # Save config in experiment directory
    with open(os.path.join(base_dir, 'config.yaml'), 'w') as f:
        yaml.dump(config, f, default_flow_style=False)
    
    return dirs


In [4]:

def setup_logging(dirs: Dict):
    """Setup logging configuration"""
    import logging
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(os.path.join(dirs['logs'], 'pipeline.log')),
            logging.StreamHandler()
        ]
    )


In [5]:

import numpy as np

def run_data_analysis(config: Dict, dirs: Dict) -> Dict:
    """Run data analysis"""
    data_config = DatasetConfig(config['data_path'])
    analyzer = DataAnalyzer(data_config)
    results = analyzer.analyze()
    
    # Save analysis results
    with open(os.path.join(dirs['logs'], 'analysis_results.json'), 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=4, cls=NumpyEncoder)
    
    return results


In [6]:

def setup_data_module(config: Dict) -> DataModule:
    """Setup data module"""
    data_config = DatasetConfig(config['data_path'])
    data_module = DataModule(
        data_config=data_config,
        model_type=ModelType.ENSEMBLE,
        batch_size=config['training']['batch_size'],
        num_workers=config['training']['num_workers'],
        image_size=config['model']['image_size']
    )
    
    return data_module


In [7]:

def create_model(config: Dict, device: torch.device) -> torch.nn.Module:
    """Create ensemble model"""
    model = create_ensemble_model(config['model'])
    return model.to(device)


In [8]:
import tqdm

def train_model(
    model: torch.nn.Module,
    data_module: DataModule,
    config: Dict,
    dirs: Dict,
    device: torch.device,
    experiment_name: str
) -> str:
    """Train model and return path to best checkpoint"""
    wandb.init(
        project="ensemble-constellation",
        name=experiment_name,
        config=config,
    )

    trainer = EnsembleTrainer(
        model=model,
        data_module=data_module,
        config=config,
        device=device,
        save_dir=dirs['checkpoints'],
        experiment_name=experiment_name
    )

    trainer.train(num_epochs=config['training']['num_epochs'])

    return os.path.join(dirs['checkpoints'], 'best_model.pt')

In [9]:

from data_preprocessing import DatasetType


def evaluate_model(
    model_path: str,
    data_module: DataModule,
    config: Dict,
    dirs: Dict,
    device: torch.device
) -> Dict:
    """Evaluate trained model"""
    model = create_ensemble_model(config['model'])
    model.load_state_dict(torch.load(model_path, map_location=device)['model_state_dict'])
    model.to(device)
    evaluator = EnsembleEvaluator(
        model=model,
        data_module=data_module,
        device=device,
        class_names=data_module.datasets[DatasetType.TEST].class_columns,
        save_dir=dirs['evaluation']
    )
    
    return evaluator.evaluate()


In [26]:
def create_summary(results: Dict, experiment_name: str) -> str:
    """Create pipeline summary"""
    summary = f"# Ensemble Model Pipeline Summary\n\n"
    summary += f"Experiment Name: {experiment_name}\n"
    summary += f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n"
    
    # Data Analysis Summary
    summary += "## Data Analysis\n"
    analysis = results['analysis_results']
    summary += f"- Total Images: {analysis['basic_stats']['total_images']}\n"
    summary += f"- Number of Classes: {analysis['basic_stats']['num_classes']}\n"
    summary += f"- Average Labels per Image: {analysis['basic_stats']['avg_labels_per_image']:.2f}\n\n"
    
    # Model Performance
    eval_results = results['evaluation_results']
    summary += "## Model Performance\n"
    summary += "\n### Ensemble Model\n"
    metrics = eval_results['ensemble_metrics']['overall_metrics']
    summary += f"- Mean Average Precision: {metrics['mean_average_precision']:.4f}\n"
    summary += f"- Exact Match Accuracy: {metrics['exact_match']:.4f}\n"
    summary += f"- Hamming Loss: {metrics['hamming_loss']:.4f}\n\n"
    
    # Model Weights
    weights = eval_results['model_weights']
    summary += "### Ensemble Weights\n"
    summary += f"- CNN Weight: {weights['cnn_weight']:.4f}\n"
    summary += f"- ViT Weight: {weights['vit_weight']:.4f}\n\n"
    
    return summary

def save_pipeline_results(
    results: Dict,
    dirs: Dict,
    experiment_name: str
):
    """Save final pipeline results"""
    # Convert numpy types to Python native types
    def convert_to_serializable(obj):
        import numpy as np
        
        if isinstance(obj, dict):
            return {key: convert_to_serializable(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [convert_to_serializable(item) for item in obj]
        elif isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
                              np.int16, np.int32, np.int64, np.uint8,
                              np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, np.float64)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        elif isinstance(obj, np.bool_):
            return bool(obj)
        return obj

    # Convert results to serializable format
    serializable_results = convert_to_serializable(results)

    # Save complete results
    with open(os.path.join(dirs['evaluation'], 'pipeline_results.json'), 'w') as f:
        json.dump(serializable_results, f, indent=4)
    
    # Create summary markdown
    summary = create_summary(results, experiment_name)
    with open(os.path.join(dirs['evaluation'], 'summary.md'), 'w') as f:
        f.write(summary)


In [11]:
config_path = 'config.yaml'
experiment_name = 'ensemble_model'

"""Run complete pipeline"""
config = load_config(config_path)
experiment_name = experiment_name or f"ensemble_{datetime.now():%Y%m%d_%H%M%S}"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dirs = setup_directories(experiment_name, config)
setup_logging(dirs)
print(dirs)


Creating experiment directory at E:\University\CU_Classes\Year-1\Fall 2024\CSCI_5502_Data Mining\Milestone_project\stellar_mapping\src\analytics\image_analytics\models\Ensemble\experiments\constellation_ensemble\ensemble_model
{'checkpoints': 'E:\\University\\CU_Classes\\Year-1\\Fall 2024\\CSCI_5502_Data Mining\\Milestone_project\\stellar_mapping\\src\\analytics\\image_analytics\\models\\Ensemble\\experiments\\constellation_ensemble\\ensemble_model\\checkpoints', 'evaluation': 'E:\\University\\CU_Classes\\Year-1\\Fall 2024\\CSCI_5502_Data Mining\\Milestone_project\\stellar_mapping\\src\\analytics\\image_analytics\\models\\Ensemble\\experiments\\constellation_ensemble\\ensemble_model\\evaluation', 'logs': 'E:\\University\\CU_Classes\\Year-1\\Fall 2024\\CSCI_5502_Data Mining\\Milestone_project\\stellar_mapping\\src\\analytics\\image_analytics\\models\\Ensemble\\experiments\\constellation_ensemble\\ensemble_model\\logs', 'visualizations': 'E:\\University\\CU_Classes\\Year-1\\Fall 2024\\CS

In [12]:
analysis_results = run_data_analysis(config, dirs)


2024-11-22 14:12:45,514 - INFO - Loaded train data: 1641 samples
2024-11-22 14:12:45,517 - INFO - Loaded valid data: 469 samples
2024-11-22 14:12:45,519 - INFO - Loaded test data: 234 samples
2024-11-22 14:12:45,519 - INFO - Starting dataset analysis...
2024-11-22 14:12:47,833 - INFO - Analysis completed


In [13]:
data_module = setup_data_module(config)

e:\University\CU_Classes\Year-1\Fall 2024\CSCI_5502_Data Mining\Milestone_project\stellar_mapping\src\analytics\image_analytics\models\Ensemble\data_preprocessing.py:203: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return torch.FloatTensor(weights)


In [14]:
model = create_model(config, device)

c:\Users\sivap\anaconda3\envs\stellar\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sivap\anaconda3\envs\stellar\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [15]:
import time
start = time.time()
best_model_path = train_model(model, data_module, config, dirs, device, experiment_name)
end = time.time()

print(f"Training time: {end - start:.2f} seconds")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: arsive. Use `wandb login --relogin` to force relogin


2024-11-22 14:12:52,008 - INFO - Starting training for 10 epochs
2024-11-22 14:12:52,008 - INFO - Training device: cuda
2024-11-22 14:12:52,009 - INFO - Save directory: E:\University\CU_Classes\Year-1\Fall 2024\CSCI_5502_Data Mining\Milestone_project\stellar_mapping\src\analytics\image_analytics\models\Ensemble\experiments\constellation_ensemble\ensemble_model\checkpoints\ensemble_model
2024-11-22 14:12:52,009 - INFO - 
Epoch 1/10
2024-11-22 14:14:00,906 - INFO - 
Training metrics:
2024-11-22 14:14:00,907 - INFO - Loss: 1.1353
2024-11-22 14:14:00,908 - INFO - Mean AP: 0.5553
2024-11-22 14:14:23,304 - INFO - 
Validation metrics:
2024-11-22 14:14:23,305 - INFO - Loss: 0.4962
2024-11-22 14:14:23,306 - INFO - Mean AP: 0.6337
2024-11-22 14:14:24,743 - INFO - 
Saved best model with validation MAP: 0.6337
2024-11-22 14:14:26,164 - INFO - 
Ensemble weights:
2024-11-22 14:14:26,166 - INFO - CNN weight: 0.4884
2024-11-22 14:14:26,166 - INFO - ViT weight: 0.5116
2024-11-22 14:14:26,167 - INFO - 


epoch,▁▂▃▃▄▅▆▆▇█
learning_rate/cnn,▁▁▁▁▁▁▁▁▁▁
learning_rate/ensemble,▁▁▁▁▁▁▁▁▁▁
learning_rate/vit,▁▁▁▁▁▁▁▁▁▁
model/cnn_weight,█▇▆▅▄▃▃▂▂▁
model/vit_weight,▁▂▃▄▅▆▆▇▇█
train/class/ aquila/ap,▁▅▆▇▇▇████
train/class/ aquila/f1,▁▄▆▆▇▇▇▇██
train/class/ aquila/precision,▁▄▆▇▇▇████
train/class/ aquila/recall,▁▃▄▄▆▅▆▆█▇
train/class/ bootes/ap,▁▃▄▆▆▇▆███


Training time: 952.03 seconds


In [ ]:
best_model_path = r"src/analytics/image_analytics/models/Ensemble/experiments/constellation_ensemble/ensemble_model/checkpoints/ensemble_model/best_model.pt"

In [ ]:

torch.save(model.state_dict(), 'src/analytics/image_analytics/models/Ensemble/experiments/constellation_ensemble/ensemble_model/checkpoints/ensemble_model/torch_best_model.pt')


In [17]:

evaluation_results = evaluate_model(best_model_path, data_module=data_module, config=config, dirs=dirs, device=device)


c:\Users\sivap\anaconda3\envs\stellar\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sivap\anaconda3\envs\stellar\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\sivap\AppData\Local\Temp\ipykernel_75088\1074215268.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytor

In [18]:

results = {
    'analysis_results': analysis_results,
    'evaluation_results': evaluation_results,
    'dirs': dirs,
    'best_model_path': best_model_path
}


In [23]:
results

{'analysis_results': {'basic_stats': {'total_images': 2344,
   'num_classes': 16,
   'split_sizes': {'train': 1641, 'valid': 469, 'test': 234},
   'avg_labels_per_image': 2.991468616697136,
   'min_labels_per_image': 0,
   'max_labels_per_image': 8},
  'class_distribution': {'train': {'class_counts': {' aquila': 255,
     ' bootes': 257,
     ' canis_major': 197,
     ' canis_minor': 233,
     ' cassiopeia': 505,
     ' cygnus': 397,
     ' gemini': 246,
     ' leo': 219,
     ' lyra': 396,
     ' moon': 372,
     ' orion': 345,
     ' pleiades': 437,
     ' sagittarius': 180,
     ' scorpius': 198,
     ' taurus': 256,
     ' ursa_major': 416},
    'class_ratios': {' aquila': 1.9803921568627452,
     ' bootes': 1.9649805447470816,
     ' canis_major': 2.563451776649746,
     ' canis_minor': 2.167381974248927,
     ' cassiopeia': 1.0,
     ' cygnus': 1.2720403022670026,
     ' gemini': 2.0528455284552845,
     ' leo': 2.3059360730593608,
     ' lyra': 1.2752525252525253,
     ' moon': 

In [27]:

save_pipeline_results(results, dirs, experiment_name)
